In [ ]:
pip install pandas

In [11]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [12]:
df0 = pd.read_csv('data/0_preprocessed.csv')
df0

,timestamp,duration,song,artist,album,song_artist
0,2019-07-05 03:00:26,2.45,7 rings,Ariana Grande,"thank u, next",7 rings - Ariana Grande
1,2019-07-05 03:01:44,1.29,All Falls Down,Alan Walker,All Falls Down,All Falls Down - Alan Walker
2,2019-07-05 03:05:23,3.67,I Don't Care,Ed Sheeran,I Don't Care,I Don't Care - Ed Sheeran
3,2019-07-05 03:08:41,3.30,Beautiful People,Ed Sheeran,Beautiful People,Beautiful People - Ed Sheeran
4,2019-07-05 03:12:01,3.32,All Falls Down,Alan Walker,All Falls Down,All Falls Down - Alan Walker
...,...,...,...,...,...,...
47672,2024-09-04 13:36:23,3.15,The Lazy Song,Bruno Mars,Doo-Wops & Hooligans,The Lazy Song - Bruno Mars
47673,2024-09-04 14:35:32,2.64,Part Of Me,Katy Perry,Teenage Dream: The Complete Confection,Part Of Me - Katy Perry
47674,2024-09-05 13:50:59,0.00,Beautiful In White,Shane Filan,Love Always,Beautiful In White - Shane Filan
47675,2024-09-05 16:08:47,0.07,Beautiful In White,Shane Filan,Love Always,Beautiful In White - Shane Filan


In [34]:
from datetime import datetime, timedelta

start_date = datetime(2019, 7, 5)
end_date = datetime(2024, 8, 31)

timestamps = []
current_date = start_date
while current_date <= end_date:
  timestamps.append(current_date)
  current_date += timedelta(weeks=1)

len(timestamps), timestamps[0], timestamps[-1]

(270,
 datetime.datetime(2019, 7, 5, 0, 0),
 datetime.datetime(2024, 8, 30, 0, 0))

In [39]:
waterfall_duration = [] # Sum of activity every 7 days
waterfall_x = [] # Timestamp
waterfall_y = [] # Change of activity every 7 days
waterfall_text = [] # Change of activity as string text
activity = 0

for i in range(len(timestamps)):
  start_week = timestamps[i]
  end_week = start_week + timedelta(days=7)

  duration = df0[(df0['timestamp'] >= str(start_week)) & (df0['timestamp'] <= str(end_week))]['duration'].sum()
  duration = int(duration)
  y = duration - activity
  text = '+' + str(y) if y >= 0 else str(y)

  waterfall_duration.append(duration)
  waterfall_x.append(start_week)
  waterfall_y.append(y)
  waterfall_text.append(text)

  activity = duration

In [40]:
waterfall_n_weeks = []
waterfall_measure = []

for i in range(len(waterfall_duration) - 1):
  waterfall_n_weeks.append(i + 1)
  if i % 45 == 0:
    waterfall_measure.append("total")
  else:
    waterfall_measure.append("relative")

waterfall_n_weeks.append(len(waterfall_duration))
waterfall_measure.append("total")

len(waterfall_n_weeks), len(waterfall_measure)

(270, 270)

In [41]:
waterfall_df = pd.DataFrame({
    'duration': waterfall_duration,
    'x': waterfall_x,
    'y': waterfall_y,
    'text': waterfall_text,
    'n_weeks': waterfall_n_weeks,
    'measure': waterfall_measure
})

waterfall_df

,duration,x,y,text,n_weeks,measure
0,125,2019-07-05,125,+125,1,total
1,507,2019-07-12,382,+382,2,relative
2,1481,2019-07-19,974,+974,3,relative
3,542,2019-07-26,-939,-939,4,relative
4,763,2019-08-02,221,+221,5,relative
...,...,...,...,...,...,...
265,629,2024-08-02,223,+223,266,relative
266,876,2024-08-09,247,+247,267,relative
267,1231,2024-08-16,355,+355,268,relative
268,1939,2024-08-23,708,+708,269,relative


In [43]:
waterfall_df['measure'].value_counts()

,count
measure,
relative,263
total,7


In [42]:
waterfall_df['duration'].sum(), df0['duration'].sum()

(163323, 163455.15999999997)

In [44]:
waterfall_df.to_csv('8_waterfall.csv', index=False)